# Demo - Adversarial Training with MNIST

In [1]:
import os
import sys
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision.utils
from torchvision import models
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torchattacks
from torchattacks import PGD, FGSM

from models import CNN

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Load MNIST Data

In [3]:
mnist_train = dsets.MNIST(root='./data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='./data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [4]:
batch_size = 128

train_loader  = torch.utils.data.DataLoader(dataset=mnist_train,
                                           batch_size=batch_size,
                                           shuffle=False)

test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                         batch_size=batch_size,
                                         shuffle=False)

## 2. Define Model

In [5]:
model = CNN().cuda()

In [6]:
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
pgd_attack = PGD(model, eps=4/255, alpha=2/255, steps=3)

## 3. Train Model

In [8]:
num_epochs = 5

In [9]:
for epoch in range(num_epochs):

    total_batch = len(mnist_train) // batch_size
    
    for i, (batch_images, batch_labels) in enumerate(train_loader):
        X = pgd_attack(batch_images, batch_labels).cuda()
        Y = batch_labels.cuda()

        pre = model(X)
        cost = loss(pre, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print('Epoch [%d/%d], lter [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, total_batch, cost.item()))

Epoch [1/5], lter [100/468], Loss: 0.3191
Epoch [1/5], lter [200/468], Loss: 0.1680
Epoch [1/5], lter [300/468], Loss: 0.1273
Epoch [1/5], lter [400/468], Loss: 0.0754
Epoch [2/5], lter [100/468], Loss: 0.0532
Epoch [2/5], lter [200/468], Loss: 0.1300
Epoch [2/5], lter [300/468], Loss: 0.0928
Epoch [2/5], lter [400/468], Loss: 0.0119
Epoch [3/5], lter [100/468], Loss: 0.0274
Epoch [3/5], lter [200/468], Loss: 0.1068
Epoch [3/5], lter [300/468], Loss: 0.0527
Epoch [3/5], lter [400/468], Loss: 0.0088
Epoch [4/5], lter [100/468], Loss: 0.0211
Epoch [4/5], lter [200/468], Loss: 0.0898
Epoch [4/5], lter [300/468], Loss: 0.0334
Epoch [4/5], lter [400/468], Loss: 0.0083
Epoch [5/5], lter [100/468], Loss: 0.0223
Epoch [5/5], lter [200/468], Loss: 0.0858
Epoch [5/5], lter [300/468], Loss: 0.0279
Epoch [5/5], lter [400/468], Loss: 0.0024


## 4. Test Model

### 4.1 Clean Accuracy

In [10]:
model.eval()

correct = 0
total = 0

for images, labels in test_loader:
    
    images = images.cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Accuracy of Clean images: %f %%' % (100 * float(correct) / total))

Accuracy of Clean images: 98.950000 %


### 4.2 Robust Accuracy

In [11]:
model.eval()

correct = 0
total = 0

fgsm_attack = FGSM(model, eps=4/255)

for images, labels in test_loader:
    
    images = fgsm_attack(images, labels).cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Accuracy of Adversarial images: %f %%' % (100 * float(correct) / total))

Accuracy of Adversarial images: 98.390000 %
